In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import cupy as cp
import joblib
import folium
import branca

from shapely.prepared import prep
from shapely.geometry import Point,Polygon
from shapely.ops import unary_union
from scipy import spatial

In [2]:
gdf = pd.concat([
        gpd.read_file('files/geojson/bdg.geojson'),
        gpd.read_file('files/geojson/cmh.geojson'),
        gpd.read_file('files/geojson/bdg_b.geojson'),
        gpd.read_file('files/geojson/k_bdg.geojson'),
        ])
gdf = gdf.reset_index(drop=True)
region_shape = unary_union(gdf.geometry)
poly = gpd.GeoDataFrame({'geometry':[region_shape]},crs=4326).to_crs(epsg=3857)['geometry'].values[0]

In [3]:
latmin, lonmin, latmax, lonmax = poly.bounds
prep_polygon = prep(poly)
valid_points = []
points = []

resolution = 500

for lat in np.arange(latmin, latmax, resolution):
    for lon in np.arange(lonmin, lonmax, resolution):
        points.append(Point((round(lat,4), round(lon,4))))

valid_points.extend(filter(prep_polygon.contains,points))

mesh_gpd = gpd.GeoDataFrame({'geometry':valid_points},crs=3857).to_crs(epsg=4326)

mesh_gpd = mesh_gpd.reset_index(drop=True)

In [5]:
# site_map = folium.Map(location=[-6.9301133,107.6208473],
#                       tiles="Cartodb voyager",
#                       zoom_start=11,
#                       zoom_control=False
#                       )

# for ind, row in mesh_gpd.iterrows():
#     point  = row.geometry
#     lat, lon = point.y, point.x
#     folium.CircleMarker([lat, lon], radius=0.1, 
#                          color='blue', fill=True, 
#                          fill_color='blue').add_to(site_map)


# fol = folium.Figure(width=900, height=400)

# site_map.add_to(fol)
# fol

In [ ]:
# From https://medium.com/@hishamsajid113/a-1000-ways-to-make-grids-but-heres-mine-6541e9a9d2

# Getting x and y coordinates from geometry object and using it to 
# generate tesselations.
mesh_gpd['lng_lat'] = mesh_gpd.geometry\
.apply(lambda coord: (round(coord.x,5),round(coord.y,5)))
vor = spatial.Voronoi(list(mesh_gpd['lng_lat'].values))


# Function I got off some gist from GitHub that converts infinite 
# bordering regions to finite regions.It just works? Apologies to 
# the original author, I am unable to locate the original gist and
# give due credit.

def func_voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.
    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.
    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.
    """

    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]

        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)




# Convering infinite border regions to finite regions.
regions,vertices=func_voronoi_finite_polygons_2d(vor,radius=0.07)


# Funciton that clips our resulting polygon within the
# bounds of our original polygon.
def func_intersect_vor(row):
    return region_shape.intersection(row.vor)

# Converting regions and vertices to Polygon objects
# and getting a geodataframe with the final grid.
polys = []
for region in regions:
    poly = vertices[region]
    polys.append(Polygon(poly))

vor_gpd = gpd.GeoDataFrame(polys)
vor_gpd = vor_gpd.rename(columns={
    0:'vor'
})


vor_gpd['bounded_vor'] = vor_gpd.apply(func_intersect_vor,1)
vor_gpd = gpd.GeoDataFrame(vor_gpd,geometry='bounded_vor')
vor_gpd = vor_gpd.drop(columns='vor')
vor_gpd = vor_gpd.reset_index()
vor_gpd = vor_gpd.rename(columns={'index':'vor_id'})
vor_gpd['vor_id'] = vor_gpd.vor_id.apply(lambda x: 'vor'+str(x))
vor_gpd['bounded_vor'] = vor_gpd['bounded_vor'].set_crs(epsg=4326)

In [ ]:
with open('files/pkl/scaler.pkl', 'rb') as f:
    scaler = joblib.load(f)

with open('files/pkl/poly.pkl', 'rb') as f:
    poly_features = joblib.load(f)

with open('files/pkl/model_greater_bdg.pkl', 'rb') as f:
    model = joblib.load(f)

In [ ]:
# sanity check
coor = [-6.913856, 107.599779]
df = pd.DataFrame({'lat': [coor[0]], 'lng': [coor[1]]})
test = scaler.transform(df)
test = poly_features.transform(test)
gpu_arr = cp.asarray_chkfinite(test)
model.predict(gpu_arr) * 1e4

In [ ]:
lat_series = mesh_gpd['lng_lat'].apply(lambda row: row[1])
lng_series = mesh_gpd['lng_lat'].apply(lambda row: row[0])
input_df = pd.DataFrame({'lat': lat_series, 'lng': lng_series})

In [ ]:
input_df = scaler.transform(input_df)
input_df = poly_features.transform(input_df)
gpu_arr = cp.asarray_chkfinite(input_df)
predictions = model.predict(gpu_arr) * 1e4
vor_gpd['prediction'] = predictions
vor_gpd = vor_gpd[vor_gpd['prediction'] >= 0]

In [ ]:
colormap = branca.colormap.LinearColormap(
    vmin=vor_gpd['prediction'].quantile(0.0),
    vmax=vor_gpd['prediction'].quantile(1),
    colors=['lightblue', 'yellow', 'orange', 'red', 'darkred', 'maroon', 'black'],
    caption="Price Prediction (Rp.)",
)
colormap

In [ ]:
site_map = folium.Map(location=[-6.9301133,107.6208473],
                      tiles="Cartodb voyager",
                      zoom_start=11,
                      zoom_control=False
                      )

tooltip = folium.GeoJsonTooltip(
    fields=["prediction"],
    aliases=["Prediction (Rp.): "],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

site_map.add_child(folium.GeoJson(
    vor_gpd,
    style_function=lambda r: {
        "fillColor": colormap(r["properties"]["prediction"]),
        "fillOpacity": 0.5,
        "weight": 0,
    },
    tooltip=tooltip
))

colormap.add_to(site_map)

fol = folium.Figure(width=900, height=400)

site_map.add_to(fol)
fol